In [1]:
val df = spark.read.format("csv").option("header", "false").load("C://Users//vaish//Downloads//cal_housing (1).data")
val Array(training_df,testing_df) = df.randomSplit(Array(0.8,0.2),seed=1234L)

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.14:4042
SparkContext available as 'sc' (version = 2.4.0, master = local[*], app id = local-1554776823802)
SparkSession available as 'spark'


df: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 7 more fields]
training_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_c0: string, _c1: string ... 7 more fields]
testing_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_c0: string, _c1: string ... 7 more fields]


In [2]:
import org.apache.spark.sql.types._
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.{Matrix, Vectors}
import org.apache.spark.ml.feature.StandardScaler
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.sql.DataFrame//so that spark knows the Dataframe type, without this, spark won't know implicit type for input
import org.apache.spark.ml.{Pipeline, PipelineModel}


def prepareData(df: DataFrame): DataFrame = {
    val new_df = df.toDF("Longitude","Latitude","MedianAge","TotalRooms","TotalBedrooms","Population","Households","MedianIncome","MedianHomeValue")
    val df2 = new_df.schema.fields.foldLeft(new_df){ 
      (df, s) => df.withColumn(s.name, df(s.name).cast(FloatType))     
    }
    val df1 = df2.withColumn("MedianHomeValue",$"MedianHomeValue"/100000)
    val df_with_features = df1.withColumn("RoomsPerHouse", col("TotalRooms")/col("Households"))
       .withColumn("PeoplePerHouse", col("Population")/col("Households"))
       .withColumn("BedroomsPerHouse", col("TotalBedrooms")/col("Households"))
    val df_analysis = df_with_features.select("MedianHomeValue","MedianAge","TotalRooms",
                  "TotalBedRooms", 
                  "Population", 
                  "Households", 
                  "MedianIncome", 
                  "RoomsPerHouse", 
                  "PeoplePerHouse", 
                  "BedroomsPerHouse")
    df_analysis
}

import org.apache.spark.sql.types._
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.{Matrix, Vectors}
import org.apache.spark.ml.feature.StandardScaler
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.{Pipeline, PipelineModel}
prepareData: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [ ]:
//val new_df = prepareData(df)

In [ ]:
//new_df.printSchema

In [3]:
//import org.apache.spark.ml.feature
//import org.apache.spark.ml.PipelineStage
//import org.apache.spark.ml.Estimator
import org.apache.spark.ml.feature.QuantileDiscretizer
//import org.apache.spark.ml.feature.Bucketizer

import org.apache.spark.ml.feature.QuantileDiscretizer
import org.apache.spark.ml.feature.Bucketizer


In [4]:
val discretizer = new QuantileDiscretizer().setNumBuckets(10).setInputCols(Array("MedianAge",
                  "TotalRooms","TotalBedRooms", "Population", "Households", "MedianIncome", "RoomsPerHouse", "PeoplePerHouse", 
                  "BedroomsPerHouse")).setOutputCols(Array("MedianAge1","TotalRooms1","TotalBedRooms1", "Population1", "Households1", 
                  "MedianIncome1", "RoomsPerHouse1", "PeoplePerHouse1", "BedroomsPerHouse1"))

discretizer: org.apache.spark.ml.feature.QuantileDiscretizer = quantileDiscretizer_b21b05ff3bf2


In [ ]:
//val result = discretizer.fit(new_df).transform(new_df)

In [ ]:
//result.printSchema

In [ ]:
//val Array(train,test) = result.randomSplit(Array(0.8,0.2),seed=1234L)

In [5]:
val assembler1 = new VectorAssembler()
    .setInputCols(Array("MedianAge1","TotalRooms1","TotalBedRooms1", "Population1", "Households1", 
                        "MedianIncome1", "RoomsPerHouse1", "PeoplePerHouse1", "BedroomsPerHouse1"))
    .setOutputCol("features")

assembler1: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_f0ffdfe7cbfd


In [6]:
val lr  = new LinearRegression()
    .setMaxIter(10)
    .setRegParam(0.3)
    .setElasticNetParam(0.8)
    .setFeaturesCol("features")
    .setLabelCol("MedianHomeValue")

lr: org.apache.spark.ml.regression.LinearRegression = linReg_75c5e74f0ddd


In [7]:
val pipeline = new Pipeline().setStages(Array(discretizer, assembler1, lr))

pipeline: org.apache.spark.ml.Pipeline = pipeline_ff90a8ac9567


In [8]:
val model = pipeline.fit(prepareData(training_df))

2019-04-08 22:27:35 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2019-04-08 22:27:35 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


model: org.apache.spark.ml.PipelineModel = pipeline_ff90a8ac9567


In [9]:
val testing_predictions = model.transform(prepareData(testing_df))
val training_predictions = model.transform(prepareData(training_df))

testing_predictions: org.apache.spark.sql.DataFrame = [MedianHomeValue: double, MedianAge: float ... 19 more fields]
training_predictions: org.apache.spark.sql.DataFrame = [MedianHomeValue: double, MedianAge: float ... 19 more fields]


In [12]:
val evaluator = new RegressionEvaluator()
  .setLabelCol("MedianHomeValue")
  .setPredictionCol("prediction")
  .setMetricName("rmse")
val rmse_train = evaluator.evaluate(training_predictions)//Training RMSE
val rmse_test = evaluator.evaluate(testing_predictions)//Testing RMSE

evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_6b3cfd482b64
rmse_train: Double = 0.9098969671559932
rmse_test: Double = 0.9193915412322796


<h1>so the rmse for training dataset is  0.9098969671559932, the rmse for testing dataset is 0.9193915412322796</h1>

<h1> model intercept and coefficients </h1>

In [13]:
import org.apache.spark.ml.regression.LinearRegressionModel
val lrModel = model.stages(2).asInstanceOf[LinearRegressionModel]
lrModel.coefficients//model coefficients

import org.apache.spark.ml.regression.LinearRegressionModel
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = linReg_75c5e74f0ddd
res0: org.apache.spark.ml.linalg.Vector = [0.0,0.0,0.0,0.0,0.0,0.16306051560031795,0.0,-0.019166800065469336,0.0]


In [14]:
model.stages(2)

res1: org.apache.spark.ml.Transformer = linReg_75c5e74f0ddd


In [15]:
lrModel.intercept//model uintercept

res2: Double = 1.4172017005389426
